# 10 Species Monkey Image Classification

The uses for image classification are endless.  This project outlines the application of image classification to zoology, specifically for 10 species of Monkeys.  This type of modelling would allow scientists to observe animals in the wild. On a sophisticated level it would allow the ability to identify individual animals. This opens up the possibility of tracking specific groups and individuals without invasive marking or tagging.  

|Label|Common Name|Latin Name|
|---|---|---|
|n0|Mantled Howler|Alouatta Palliata|
|n1|Patas Monkey|Erythrocebus Pata|
|n2|Bald Uakari|Cacajao Calvus|
|n3|Japanese Macaque|Macaca Fusacata|
|n4|Pygmy Marmoset|Cebuella Pygmea|
|n5|White Headed Capuchin|Cebus Capucinus|
|n6|Silvery Marmoset|Mico Argentatus|
|n7|Common Squirrel Monkey|Saimiri Sciureus|
|n8|Black Headed Night Monkey|Aotus Nigriceps|
|n9|Nilgiri Langur|Trachypithecus Johnii|

## The Data

The original data can be found at : https://www.kaggle.com/slothkong/10-monkey-species

This data set contains over 1000 images of 10 different monkey speicies.  It was originally divided into training and testing.  For the purposes of validation I extracted 12 photos of each species from the training file (the largest), and organized them into folders by species.  The corresponding numbers with species is displayed in the table above.  These files are on my local desktop and would need to be extracted from the website to the local server with corresponding paths.

In [ ]:
#Necessary Imports
#import os
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Tensor Imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from tensorflow.keras import models, layers, optimizers

#importing confusion matrix/ classification report
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import classification_report



### Data Visualization

The purpose of this section was solely to demonstrate that the images are properly fed into Jupyter Notebook with its corresponding label.  This prevents having to add labels into the coding. 

In [ ]:
# Create objects to use for directory paths.

n0_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n0'
n1_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n1'
n2_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n2'
n3_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n3'
n4_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n4'
n5_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n5'
n6_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n6'
n7_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n7'
n8_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n8'
n9_images = '/Users/andrewozbun/Desktop/Monkey_Images/training/n9'

In [ ]:
# Directory with Mantled Howler.
n0_dir = os.path.join(n0_images)

# Directory with Patas Monkey.
n1_dir = os.path.join(n1_images)

# Directory with Bald Uakari.
n2_dir = os.path.join(n2_images)

# Directory with Japanese Macaque.
n3_dir = os.path.join(n3_images)

# Directory with Pygmy Marmoset.
n4_dir = os.path.join(n4_images)

# Directory with White Headed Capuchin.
n5_dir = os.path.join(n5_images)

# Directory with Silvery Marmoset.
n6_dir = os.path.join(n6_images)

# Directory with Common Squirrel Monkey.
n7_dir = os.path.join(n7_images)

# Directory with Black Headed Night Monkey.
n8_dir = os.path.join(n8_images)

# Directory with Nilgiri Langur
n9_dir = os.path.join(n9_images)

In [ ]:
#Looking at the amount of images in each class to make sure that the path is working correctly

print('total Mantled Howler images:', len(os.listdir(n0_dir)))
print('total Patas Monkey images:', len(os.listdir(n1_dir)))
print('total Bald Uakari images:', len(os.listdir(n2_dir)))
print('total Japanese Macaque images:', len(os.listdir(n3_dir)))
print('total Pygmy Marmoset images:', len(os.listdir(n4_dir)))
print('total White Headed Capuchin images:', len(os.listdir(n5_dir)))
print('total Silvery Marmoset images:', len(os.listdir(n6_dir)))
print('total Common Squirrl Monkey images:', len(os.listdir(n7_dir)))
print('total Black Headed Night Monkey images:', len(os.listdir(n8_dir)))
print('total Nilgiri Langur images:', len(os.listdir(n9_dir)))

In [ ]:
train_n0 = os.listdir(n0_dir)
print(train_n0[:5])

train_n1 = os.listdir(n1_dir)
print(train_n1[:5])

In [ ]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

In [ ]:
# Index for iterating over images
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_n0_pix = [os.path.join(n0_dir, fname) 
                for fname in train_n0[pic_index-8:pic_index]]
next_n1_pix = [os.path.join(n1_dir, fname) 
                for fname in train_n1[pic_index-8:pic_index]]
print ("Mantled Howler")
print()
for i, img_path in enumerate(next_n0_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

print ("Patas Monkey")
print()
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)
for i, img_path in enumerate(next_n1_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

## Building the Models

**MLP v CNN**

Multilayer Perceptrons, or MLPs for short, are the classical type of neural network.
They are comprised of one or more layers of neurons. Data is fed to the input layer, there may be one or more hidden layers providing levels of abstraction, and predictions are made on the output layer, also called the visible layer.


Convolutional Neural Networks, or CNNs, were designed to map image data to an output variable.
They have proven so effective that they are the go-to method for any type of prediction problem involving image data as an input.

### MLP

**Back-propagation** is just a way of propagating the total loss back into the neural network to know how much of the loss every node is responsible for, and subsequently updating the weights in such a way that minimizes the loss by giving the nodes with higher error rates lower weights and vice versa.

#### Data Preprocessing

In [ ]:
#set as objects
train_path = '/Users/andrewozbun/Desktop/Monkey_Images/training'
test_path = '/Users/andrewozbun/Desktop/Monkey_Images/testing'
validation_path = '/Users/andrewozbun/Desktop/Monkey_Images/validation'

train_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(train_path, target_size=(225, 225), 
                                                         classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9'],
                                                         batch_size=25)
test_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(test_path, target_size=(225, 225), 
                                                         classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9'],
                                                         batch_size=25)
validation_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(validation_path, target_size=(225, 225), 
                                                         classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9'],
                                                         batch_size=25)


#### Buidling a baseline Model

In [ ]:
# Classification MLP(Multilayer perceptron) 
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(350, activation=keras.activations.relu),
    keras.layers.Dense(50, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.softmax)
])


In [ ]:
#Model
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=train_batches,validation_data= validation_batches, epochs=10, verbose=1)
          

### CNN

In [ ]:
cnn_model = tf.keras.models.Sequential([
    # The input shape is the desired size of the image 225x 225 with 3 bytes color, RGB.
    #This is also the input "layer", the first convolution is the first hidden layer.
    # The first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(225, 225, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #first number is nodes or filters, second number is the kernel size.
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Relu or Rectified linear Unit,
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    

    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 10 output neurons for 10 classes with the softmax activation
    tf.keras.layers.Dense(10, activation='softmax')
])


In [ ]:
cnn_model.summary

In [ ]:
cnn_model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = cnn_model.fit(train_batches, validation_data= validation_batches, shuffle = False, epochs=30, batch_size=20, verbose=1)

In [ ]:
cnn_model.evaluate(test_batches)

In [ ]:
cnn_model.predict(test_batches)

In [ ]:
classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9']

In [ ]:
model_prediction = cnn_model.predict_classes(test_batches)
model_prediction

In [ ]:
np.array(classes)[model_prediction]

In [ ]:
pred = cnn_model.predict(test_batches)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(30)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_batches.classes

In [ ]:
predictions = cnn_model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
print(cm)

In [ ]:
def plot_confusion_matrix(cm, classes, 
                         normalize = False,
                         fontsize = 20,
                         cmap=plt.cm.Blues):

    plt.figure(figsize=(30, 30))
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.colorbar
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation =45)
    plt.yticks(tick_marks, classes)
    plt.tick_params(axis='both', which='major', labelsize=24)
    
    label_font = {'size':'30'}  
    title_font = {'size':'35'}
    
    thresh = cm.max()/ 2
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(i,j, cm[i,j],
        horizontalalignment='center',
        fontsize=fontsize,
        color='black')

    plt.tight_layout()
    plt.ylabel('True Label',fontdict=label_font)
    plt.xlabel('Predicted Label',fontdict=label_font)
    plt.title('Confusion Matrix',fontdict=title_font)

In [ ]:
test_batches.class_indices

In [ ]:
plot_confusion_matrix(cm = cm, classes=classes)

In [ ]:
print(classification_report(y_true=test_batches.classes,  y_pred=np.argmax(predictions, axis =-1)))

#### Augmentation

In [ ]:
aug_cnn = tf.keras.models.Sequential([
    # The input shape is the desired size of the image 225x 225 with 3 bytes color, RGB.
    #This is also the input "layer", the first convolution is the first hidden layer.
    # The first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(225, 225, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #first number is nodes or filters, second number is the kernel size.
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Relu or Rectified linear Unit,
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 10 output neurons for 10 classes with the softmax activation
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=('adam'), metrics = 'accuracy')

In [ ]:
#Augmenting the Data
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
#created an opbject that stores train_datagen.flow
train_generator = train_datagen.flow_from_directory(train_path, target_size=(225, 225), 
                                                         classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9'],
                                                         batch_size=32)

validation_generator = validation_datagen.flow_from_directory(validation_path, target_size=(225, 225), 
                                                         classes = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9'],
                                                         batch_size=32)

In [ ]:
aug_history = aug_cnn.fit(
    train_datagen.flow(train_batches),
    epochs=40,
    validation_data=validation_batches,
    verbose=2)

#### Regularization

### VGG16 Model

The award winning VGG16 model was developed at Oxford in 2014 by the Visual Geometry Group.  Hence, its name.  The model was trained on 1000 different classes with over 10 million parameters and 95% accuracy.  It has become a sought after model ever since.

#### Transfer Learning

Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task.  Transfer learning can be used to speed up the learning process and heighten the accuracy.  

In [ ]:
vgg_model = VGG16()
print(vgg_model.summary())

In [ ]:
type(vgg_model)

In [ ]:
model_base = VGG16(include_top=False)

x = model_base.output
predictions = layers.Dense(10, activation='softmax')(x)

Vgg16_model = models.Model(inputs= model_base.input, outputs=predictions)

In [ ]:
Vgg16_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
Vgg16_model.summary()

In [ ]:
history_vgg = Vgg16_model.fit(train_batches, 
                              validation_data=validation_batches, 
                              epochs=10, verbose=1,
                              steps_per_epoch=len(train_batches)/32, 
                              validation_steps=len(validation_batches)/32)